In [1]:
from pathlib import Path

import numpy as np
import pandas as pd


pd.set_option("display.max_columns", 100)

from google.colab import drive
drive.mount('/content/gdrive')
DATA_PATH = "/content/gdrive/My Drive/MLdata/"

train_df = pd.read_csv(
    DATA_PATH + "train.csv", 
    index_col="tripid"
)
test_df = pd.read_csv(
    DATA_PATH + "test.csv", 
    index_col="tripid"
)


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
train_df["label"] = train_df["label"].map({"incorrect": 0, "correct":1})

In [3]:
train_df["pickup"] = pd.to_datetime(train_df["pickup_time"])
train_df["drop"] = pd.to_datetime(train_df["drop_time"])
train_df ['difference'] = train_df["drop"] - train_df["pickup"]
train_df ['difference'] = train_df ['difference']/np.timedelta64(1,'s')
train_df['duration'] = train_df['duration'].fillna(train_df ['difference'])
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17176 entries, 189123628 to 213817296
Data columns (total 16 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   additional_fare            16974 non-null  float64       
 1   duration                   17176 non-null  float64       
 2   meter_waiting              16974 non-null  float64       
 3   meter_waiting_fare         16974 non-null  float64       
 4   meter_waiting_till_pickup  16974 non-null  float64       
 5   pickup_time                17176 non-null  object        
 6   drop_time                  17176 non-null  object        
 7   pick_lat                   17176 non-null  float64       
 8   pick_lon                   17176 non-null  float64       
 9   drop_lat                   17176 non-null  float64       
 10  drop_lon                   17176 non-null  float64       
 11  fare                       17039 non-null  float64     

In [4]:
train_df.head()
train_df.isnull().sum()
train_df.head()

,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,pick_lat,pick_lon,drop_lat,drop_lon,fare,label,pickup,drop,difference
tripid,,,,,,,,,,,,,,,,
189123628,10.5,834.0,56.0,0.0000,64.0,11/1/2019 0:20,11/1/2019 0:34,6.86252,79.8993,6.90330,79.8783,270.32,1,2019-11-01 00:20:00,2019-11-01 00:34:00,840.0
189125358,10.5,791.0,47.0,0.0000,134.0,11/1/2019 0:56,11/1/2019 1:09,6.88589,79.8984,6.91373,79.8923,197.85,1,2019-11-01 00:56:00,2019-11-01 01:09:00,780.0
189125719,10.5,1087.0,80.0,0.0000,61.0,11/1/2019 1:08,11/1/2019 1:26,6.90839,79.8651,6.93669,79.9146,301.64,1,2019-11-01 01:08:00,2019-11-01 01:26:00,1080.0
189127273,10.5,598.0,271.0,15.6638,68.0,11/1/2019 2:27,11/1/2019 2:37,6.92570,79.8895,6.92748,79.8971,82.30,1,2019-11-01 02:27:00,2019-11-01 02:37:00,600.0
189128020,NaN,1020.0,NaN,NaN,NaN,11/1/2019 3:34,11/1/2019 3:51,6.87441,79.8615,6.84478,79.9290,358.39,1,2019-11-01 03:34:00,2019-11-01 03:51:00,1020.0


In [5]:
train_df2 = train_df.drop(['pickup_time','drop_time','pickup','drop','difference'],axis='columns')
train_df2.isnull().sum()

additional_fare              202
duration                       0
meter_waiting                202
meter_waiting_fare           202
meter_waiting_till_pickup    202
pick_lat                       0
pick_lon                       0
drop_lat                       0
drop_lon                       0
fare                         137
label                          0
dtype: int64

In [6]:
train_df2['additional_fare'] = train_df2['additional_fare'].fillna(train_df2.additional_fare.mean())
train_df2['meter_waiting'] = train_df2['meter_waiting'].fillna(train_df2.meter_waiting.mean())
train_df2['meter_waiting_fare'] = train_df2['meter_waiting_fare'].fillna(train_df2.meter_waiting_fare.mean())
train_df2['meter_waiting_till_pickup'] = train_df2['meter_waiting_till_pickup'].fillna(train_df2.meter_waiting_till_pickup.mean())
train_df2.isnull().sum()


additional_fare                0
duration                       0
meter_waiting                  0
meter_waiting_fare             0
meter_waiting_till_pickup      0
pick_lat                       0
pick_lon                       0
drop_lat                       0
drop_lon                       0
fare                         137
label                          0
dtype: int64

In [7]:
train_df2= train_df2.dropna()

In [8]:
from math import radians, cos, sin, asin, sqrt
import numpy as np

def haversine_np(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)

    All args must be of equal length.    

    """
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2

    c = 2 * np.arcsin(np.sqrt(a))
    km = 6371 * c  # 6371 is Radius of earth in kilometers. Use 3956 for miles
    return km

train_df2['distance'] = haversine_np(train_df2['pick_lat'], train_df2['pick_lon'], train_df2['drop_lat'] , train_df2['drop_lon'])

In [9]:
clean_df = train_df2.drop(['pick_lat','pick_lon','drop_lat','drop_lon'],axis='columns')

In [10]:

clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17039 entries, 189123628 to 213817296
Data columns (total 8 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   additional_fare            17039 non-null  float64
 1   duration                   17039 non-null  float64
 2   meter_waiting              17039 non-null  float64
 3   meter_waiting_fare         17039 non-null  float64
 4   meter_waiting_till_pickup  17039 non-null  float64
 5   fare                       17039 non-null  float64
 6   label                      17039 non-null  int64  
 7   distance                   17039 non-null  float64
dtypes: float64(7), int64(1)
memory usage: 1.2 MB


In [11]:
clean_df.head()

,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,fare,label,distance
tripid,,,,,,,,
189123628,10.500000,834.0,56.000000,0.000000,64.000000,270.32,1,2.467063
189125358,10.500000,791.0,47.000000,0.000000,134.000000,197.85,1,0.868942
189125719,10.500000,1087.0,80.000000,0.000000,61.000000,301.64,1,5.531798
189127273,10.500000,598.0,271.000000,15.663800,68.000000,82.30,1,0.845795
189128020,13.719651,1020.0,629.074231,32.057666,112.466832,358.39,1,7.527884


In [12]:
clean_df.isnull().sum()

additional_fare              0
duration                     0
meter_waiting                0
meter_waiting_fare           0
meter_waiting_till_pickup    0
fare                         0
label                        0
distance                     0
dtype: int64

In [13]:
clean_df = clean_df.drop(['meter_waiting_till_pickup'],axis='columns')

**Training**

In [14]:
import keras
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation,Flatten
from keras.layers import Convolution2D,MaxPooling2D,BatchNormalization,Convolution1D,Conv1D,MaxPooling1D
from keras.optimizers import SGD,RMSprop
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score

Using TensorFlow backend.


In [15]:
X_train = clean_df.drop(['label'],axis='columns')
Y_train = clean_df['label']
Y_train.head()

tripid
189123628    1
189125358    1
189125719    1
189127273    1
189128020    1
Name: label, dtype: int64

In [16]:
from sklearn.model_selection import train_test_split
x, x_test, y, y_test = train_test_split(X_train, Y_train, test_size=0.25)

In [17]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x = sc.fit_transform(x)
x_test = sc.transform(x_test)


In [19]:
from keras.layers import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU

import keras.backend as K
n_col = x.shape[1]
print(n_col)
def rmse (y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1))

def model() :   #Created a Model using Keras
    model = Sequential()
    model.add(Dropout(0.2,input_shape=(n_col,)))
    model.add(BatchNormalization())
    model.add(Dense(512,activation='relu'))#512 neurons in input layer
    model.add(Dropout(0.2))
    model.add(BatchNormalization())
    model.add(Dense(256,activation='relu')) #256 neurons in hidden layer
    model.add(BatchNormalization())
    model.add(Dense(128,activation='relu'))  # 128 neurons in hidden layer
    model.add(BatchNormalization())
    model.add(Dense(64,activation='relu'))   # 64 neurons in hidden layer
    model.add(BatchNormalization())
    model.add(Dense(32,activation='relu'))   # 32 neurons in hidden layer
    model.add(BatchNormalization())
    model.add(Dense(16,activation='relu')) # 16 neurons in hidden layer
    model.add(BatchNormalization())
    model.add(Dense(8,activation='relu')) # 8 neurons in hidden layer
    model.add(BatchNormalization())
    model.add(Dense(1)) # 1 neuron in output layer
    
    #nadam = optimizers.Nadam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=0.004)
    #adadelta =optimizers.Adadelta(lr=1.0, rho=0.95, epsilon=None, decay=0.0)
    #adgrad = optimizers.Adagrad(lr=0.001, epsilon=None, decay=0.0)
    #rms = optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0)
    
    #sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True,clipnorm=0.5)

    model.compile(optimizer='adam',loss='mse', metrics=[rmse])
    return model


model = model()
model.fit(x, y, epochs=80, batch_size=150, verbose=1, validation_data=(x_test, y_test))

6
Train on 12779 samples, validate on 4260 samples
Epoch 1/80
12779/12779 [==============================] - 3s 221us/step - loss: 0.8654 - rmse: 0.8241 - val_loss: 0.4408 - val_rmse: 0.6545
Epoch 2/80
12779/12779 [==============================] - 1s 97us/step - loss: 0.3230 - rmse: 0.5391 - val_loss: 0.1852 - val_rmse: 0.4256
Epoch 3/80
12779/12779 [==============================] - 1s 97us/step - loss: 0.1367 - rmse: 0.3293 - val_loss: 0.0926 - val_rmse: 0.2352
Epoch 4/80
12779/12779 [==============================] - 1s 95us/step - loss: 0.0822 - rmse: 0.2041 - val_loss: 0.0799 - val_rmse: 0.1374
Epoch 5/80
12779/12779 [==============================] - 1s 97us/step - loss: 0.0748 - rmse: 0.1620 - val_loss: 0.0859 - val_rmse: 0.1218
Epoch 6/80
12779/12779 [==============================] - 1s 96us/step - loss: 0.0707 - rmse: 0.1489 - val_loss: 0.0880 - val_rmse: 0.1216
Epoch 7/80
12779/12779 [==============================] - 1s 95us/step - loss: 0.0710 - rmse: 0.1472 - val_loss: 0

In [ ]:
# define the keras model
model2 = Sequential()
model2.add(Dense(256, input_dim=6, activation='relu'))
model2.add(Dropout(0.5))
model2.add(Dense(128, activation='relu'))
model2.add(Dropout(0.5))
model2.add(Dense(64, activation='relu'))
model2.add(Dropout(0.5))
model2.add(Dense(32, activation='relu'))
model2.add(Dropout(0.5))
model2.add(Dense(1, activation='sigmoid'))
sgd=SGD(lr=0.01,momentum=0.1)
# compile the keras model
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit the keras model on the dataset
model2.fit(x, y, epochs=100, batch_size=32, verbose=1, validation_data=(x_test, y_test))
# evaluate the keras model
_, accuracy = model2.evaluate(x_test, y_test)
print('Accuracy: %.2f' % (accuracy*100))

**Predicting**

In [ ]:
test_df.isnull().sum()

additional_fare              0
duration                     0
meter_waiting                0
meter_waiting_fare           0
meter_waiting_till_pickup    0
pickup_time                  0
drop_time                    0
pick_lat                     0
pick_lon                     0
drop_lat                     0
drop_lon                     0
fare                         0
dtype: int64

In [ ]:
test_df.info()
test_df.head()
test_df.isnull().sum()
test_df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8576 entries, 213284604 to 222860703
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   additional_fare            8576 non-null   float64
 1   duration                   8576 non-null   int64  
 2   meter_waiting              8576 non-null   int64  
 3   meter_waiting_fare         8576 non-null   float64
 4   meter_waiting_till_pickup  8576 non-null   int64  
 5   pickup_time                8576 non-null   object 
 6   drop_time                  8576 non-null   object 
 7   pick_lat                   8576 non-null   float64
 8   pick_lon                   8576 non-null   float64
 9   drop_lat                   8576 non-null   float64
 10  drop_lon                   8576 non-null   float64
 11  fare                       8576 non-null   float64
dtypes: float64(7), int64(3), object(2)
memory usage: 871.0+ KB


,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,pick_lat,pick_lon,drop_lat,drop_lon,fare
tripid,,,,,,,,,,,,
213284604,10.5,924,42,2.4486,148,2/1/2020 0:38,2/1/2020 0:53,6.83454,79.8750,6.77490,79.8840,289.27
213286352,10.5,4249,20,0.0000,91,2/1/2020 1:02,2/1/2020 2:13,6.91168,79.8723,6.55091,79.9706,1912.70
213293973,10.5,1552,255,2.6588,23,2/1/2020 5:02,2/1/2020 5:28,6.92145,79.8478,6.90539,79.8989,394.00
213294622,10.5,462,16,0.0000,198,2/1/2020 5:30,2/1/2020 5:38,6.77433,79.9416,6.80401,79.9407,154.32
213298687,10.5,814,392,12.3692,69,2/1/2020 7:00,2/1/2020 7:14,6.97968,79.9130,6.98875,79.8914,147.47


In [ ]:
test_df['distance'] = haversine_np(test_df['pick_lat'], test_df['pick_lon'], test_df['drop_lat'] , test_df['drop_lon'])

In [ ]:
test_df2 = test_df.drop(['pickup_time','drop_time','pick_lat','pick_lon','drop_lat','drop_lon'],axis='columns')

test_df2.isnull().sum()

In [ ]:
test_df2 = test_df2.drop(['meter_waiting_till_pickup'],axis='columns')

In [ ]:
testdf2= sc.transform(test_df2)

In [ ]:
predictions = model.predict_classes(testdf2)

In [ ]:
test_df2['prediction'] = predictions

In [ ]:
final_df = test_df2.drop(['additional_fare','distance','duration','fare','meter_waiting','meter_waiting_fare'],axis='columns')

final_df.head()
final_df.info()

In [ ]:
final_df.head(10)

,prediction
tripid,
213284604,1
213286352,1
213293973,1
213294622,1
213298687,1
213299545,1
213302332,1
213302671,1
213305594,1


In [ ]:
final_df.to_csv(DATA_PATH+'submission_30.csv', index=True)